举个例子，假设我们想根据5x5的图像来进行平滑操作，图像矩阵的布局如下。

|       |      | $x_i$ | $x_i$ | $x_i$ | $x_i$ | $x_i$ |
| ----- | ---- | ----- | ----- | ----- | ----- | ----- |
|       |      | -2    | -1    | 0     | 1     | 2     |
| $y_i$ | -2   | d(0)  | d(1)  | d(2)  | d(3)  | d(4)  |
| $y_i$ | -1   | d(5)  | d(6)  | d(7)  | d(8)  | d(9)  |
| $y_i$ | 0    | d(10) | d(11) | d(12) | d(13) | d(14) |
| $y_i$ | 1    | d(15) | d(16) | d(17) | d(18) | d(19) |
| $y_i$ | 2    | d(20) | d(21) | d(22) | d(23) | d(24) |

`d(i)` 是像素值， 列向量 `d` 代表所有图像数据：

$$
d = (\,d(0)\,d(1)\, ...\, d(24)\,)^T
$$

我们想要拟合一个 $3^{rd}$ 阶, 二维多项式来拟合这个图像：

$$
d(i) ≈ f(x_i ,y_i) = a_{00} + a_{10}x_i + a_{01}y_i + a_{20}x_i^2 + a_{11}x_iy_i + a_{02}y_i^2 + a_{30}x_i^3 + a_{21}x_i^2y_i + a_{12}x_iy_i^2 + a_{03}y_i^3
$$

Note that the coefficient of x i y j is a ij . (x i ,y i ) is the pixel coordinate of d(i).

To compute the coefficients from the data we set up a matrix equation:

Xa = d

In [1]:
import sys
import cv2
print("OpenCV version:", cv2.__version__)
import matplotlib.pyplot as plt
import matplotlib
print("Matplotlib.Pyplot", matplotlib.__version__)
import numpy as np
print("Numpy version:", np.__version__)
import math
from scipy import signal
import scipy

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

# 用于在 Jupyter Notebook 内部显示图片
%matplotlib inline    

def sgolay2d ( z, window_size, order, derivative=None):
    # number of terms in the polynomial expression
    n_terms = ( order + 1 ) * ( order + 2)  / 2.0

    if  window_size % 2 == 0:  raise ValueError('窗口大小必须是奇数')
    if window_size**2 < n_terms:  raise ValueError('阶数过高')

    half_size = window_size // 2  # 向下取整

    # 多项式各项的阶数
    # exps = [(0,0), (1,0), (0,1), (2,0), (1,1), (0,2), ...]
    exps = [ (k-n, n) for k in range(order+1) for n in range(k+1) ]
    print("Exps : ", exps)

    # 点坐标系
    ind = np.arange(-half_size, half_size+1, dtype=np.float64)
    print("ind : ", ind)
    dx = np.repeat( ind, window_size )
    print("dx : ", dx)
    dy = np.tile( ind, [window_size, 1]).reshape(window_size**2, )
    print("dy : ", dy)

    # 建立方程组的矩阵
    A = np.empty( (window_size**2, len(exps)) )
    # print("A : ", A)
    for i, exp in enumerate( exps ):
        print("i, exp", i, exp)
        A[:,i] = (dx**exp[0]) * (dy**exp[1])
    # print("A_0 : ", A)

    # 在四个边框处用适当的值填充输入阵列
    new_shape = z.shape[0] + 2*half_size, z.shape[1] + 2*half_size  # 将原来的图像向外扩展半个窗口大小
    Z = np.zeros( (new_shape) )
    
    # top band
    band = z[0, :]
    Z[:half_size, half_size:-half_size] =  band -  np.abs( np.flipud( z[1:half_size+1, :] ) - band )
    # bottom band
    band = z[-1, :]
    Z[-half_size:, half_size:-half_size] = band  + np.abs( np.flipud( z[-half_size-1:-1, :] )  -band )
    # left band
    band = np.tile( z[:,0].reshape(-1,1), [1,half_size])
    Z[half_size:-half_size, :half_size] = band - np.abs( np.fliplr( z[:, 1:half_size+1] ) - band )
    # right band
    band = np.tile( z[:,-1].reshape(-1,1), [1,half_size] )
    Z[half_size:-half_size, -half_size:] =  band + np.abs( np.fliplr( z[:, -half_size-1:-1] ) - band )
    # central band
    Z[half_size:-half_size, half_size:-half_size] = z

    # 左上角↖️
    band = z[0,0]
    Z[:half_size,:half_size] = band - np.abs( np.flipud(np.fliplr(z[1:half_size+1,1:half_size+1]) ) - band )
    # 右下角↘️
    band = z[-1,-1]
    Z[-half_size:,-half_size:] = band + np.abs( np.flipud(np.fliplr(z[-half_size-1:-1,-half_size-1:-1]) ) - band )
    # 右上角↗️
    band = Z[half_size,-half_size:]
    Z[:half_size,-half_size:] = band - np.abs( np.flipud(Z[half_size+1:2*half_size+1,-half_size:]) - band )
    # 左下角↙️
    band = Z[-half_size:,half_size].reshape(-1,1)
    Z[-half_size:,:half_size] = band - np.abs( np.fliplr(Z[-half_size:, half_size+1:2*half_size+1]) - band )

    # 解方程组并卷积
    if derivative == None:
        m = np.linalg.pinv(A)[0].reshape((window_size, -1))
        return scipy.signal.fftconvolve(Z, m, mode='valid')
    
    elif derivative == 'col':
        c = np.linalg.pinv(A)[1].reshape((window_size, -1))
        return scipy.signal.fftconvolve(Z, -c, mode='valid')
    
    elif derivative == 'row':
        r = np.linalg.pinv(A)[2].reshape((window_size, -1))
        return scipy.signal.fftconvolve(Z, -r, mode='valid')
    
    elif derivative == 'both':
        c = np.linalg.pinv(A)[1].reshape((window_size, -1))
        r = np.linalg.pinv(A)[2].reshape((window_size, -1))
        return scipy.signal.fftconvolve(Z, -r, mode='valid'), scipy.signal.fftconvolve(Z, -c, mode='valid')

OpenCV version: 4.5.5
Matplotlib.Pyplot 3.5.1
Numpy version: 1.21.2


In [2]:
rand = np.random.randn(5, 5)
Mat = np.arange(1, 26, 1).reshape(5, 5) + rand
print(Mat)


Mat_s = sgolay2d(Mat, 5, 3)
print(Mat_s)

[[ 0.81754339  2.54477609  3.54301469  4.66806967  5.72925732]
 [ 7.79662514  7.20935075  6.93045546  8.1603216  10.29581417]
 [10.91129607 11.48704647 11.92877612 13.01426737 15.26782744]
 [17.00441106 16.85777536 17.73151798 19.66099882 21.14961806]
 [21.92778149 22.27814628 22.74506797 23.58504288 24.7221467 ]]
Exps :  [(0, 0), (1, 0), (0, 1), (2, 0), (1, 1), (0, 2), (3, 0), (2, 1), (1, 2), (0, 3)]
ind :  [-2. -1.  0.  1.  2.]
dx :  [-2. -2. -2. -2. -2. -1. -1. -1. -1. -1.  0.  0.  0.  0.  0.  1.  1.  1.
  1.  1.  2.  2.  2.  2.  2.]
dy :  [-2. -1.  0.  1.  2. -2. -1.  0.  1.  2. -2. -1.  0.  1.  2. -2. -1.  0.
  1.  2. -2. -1.  0.  1.  2.]
i, exp 0 (0, 0)
i, exp 1 (1, 0)
i, exp 2 (0, 1)
i, exp 3 (2, 0)
i, exp 4 (1, 1)
i, exp 5 (0, 2)
i, exp 6 (3, 0)
i, exp 7 (2, 1)
i, exp 8 (1, 2)
i, exp 9 (0, 3)
[[ 0.98797742  2.27733793  3.63271419  4.64625101  5.72925732]
 [ 6.17784449  6.82642658  7.78409067  8.84504    10.28734007]
 [11.6909991  11.92035267 12.45789052 13.72025457 15.5869726 ]

In [3]:
c = np.arange(1, 10)
print(c)
e = np.repeat(c, 3)
print(e)
d = np.tile(c, 3)
print(d)
c = np.tile(c, [3, 1])
print(c)


[1 2 3 4 5 6 7 8 9]
[1 1 1 2 2 2 3 3 3 4 4 4 5 5 5 6 6 6 7 7 7 8 8 8 9 9 9]
[1 2 3 4 5 6 7 8 9 1 2 3 4 5 6 7 8 9 1 2 3 4 5 6 7 8 9]
[[1 2 3 4 5 6 7 8 9]
 [1 2 3 4 5 6 7 8 9]
 [1 2 3 4 5 6 7 8 9]]


In [4]:
c

array([[1, 2, 3, 4, 5, 6, 7, 8, 9],
       [1, 2, 3, 4, 5, 6, 7, 8, 9],
       [1, 2, 3, 4, 5, 6, 7, 8, 9]])

In [6]:
c.shape

(3, 9)

In [5]:
test = np.zeros((9, 9))
print(np.flipud( test[1:3, :] ) )

[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]
